In [1]:
import pandas as pd

data = pd.read_csv("datasets/test.csv")
oil = pd.read_csv("datasets/oil.csv")
holidays = pd.read_csv("datasets/holidays_events.csv")
stores = pd.read_csv("datasets/stores.csv")
transactions = pd.read_csv("datasets/transactions.csv")

In [2]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [3]:
holidays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [4]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [5]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [6]:
data['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [7]:
date_range = pd.date_range('2013-01-01', '2017-08-31')
calendar = pd.DataFrame(date_range,columns=['date'])
calendar

,date
0,2013-01-01
1,2013-01-02
2,2013-01-03
3,2013-01-04
4,2013-01-05
...,...
1699,2017-08-27
1700,2017-08-28
1701,2017-08-29
1702,2017-08-30


In [8]:
oil['date'] = pd.to_datetime(oil['date'] , errors = 'coerce')
calendar = pd.merge(calendar, oil, how='left', on='date')

calendar['dcoilwtico'].fillna(method='ffill', inplace=True)
calendar['dcoilwtico'].fillna(method='bfill', inplace=True)

/var/folders/y_/tqdmcpj93n34bc271lhysl0w0000gn/T/ipykernel_58455/2249823845.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  calendar['dcoilwtico'].fillna(method='ffill', inplace=True)
/var/folders/y_/tqdmcpj93n34bc271lhysl0w0000gn/T/ipykernel_58455/2249823845.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  calendar['dcoilwtico'].fillna(method='bfill', inplace=True)


In [9]:
holidays['date'] = holidays['date'].replace({'2013-04-29' : pd.to_datetime('2013-03-29')})

holidays = holidays[holidays.locale == 'National'] 

In [10]:
holidays['date'] = pd.to_datetime(holidays['date'] , errors = 'coerce')
holidays = holidays.drop_duplicates(subset=['date'])

In [11]:
calendar['day_week'] = calendar['date'].dt.dayofweek

calendar['holiday_or_weekday'] = 'weekday'
calendar.loc[calendar['day_week'] > 4, 'holiday_or_weekday'] = 'horiday'

calendar.head()

,date,dcoilwtico,day_week,holiday_or_weekday
0,2013-01-01,93.14,1,weekday
1,2013-01-02,93.14,2,weekday
2,2013-01-03,92.97,3,weekday
3,2013-01-04,93.12,4,weekday
4,2013-01-05,93.12,5,horiday


In [12]:
calendar = pd.merge(calendar,holidays,how='left',on='date')
calendar.head()

,date,dcoilwtico,day_week,holiday_or_weekday,type,locale,locale_name,description,transferred
0,2013-01-01,93.14,1,weekday,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-02,93.14,2,weekday,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,92.97,3,weekday,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,93.12,4,weekday,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,93.12,5,horiday,Work Day,National,Ecuador,Recupero puente Navidad,False


In [13]:
calendar.loc[calendar.type == 'Bridge'  , 'holiday_or_weekday'] = 'holiday'
calendar.loc[calendar.type == 'Transfer', 'holiday_or_weekday'] = 'holiday'

calendar.loc[calendar.type == 'Work Day', 'holiday_or_weekday'] = 'weekday'


calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == False), 'holiday_or_weekday'] = 'holiday'
calendar.loc[(calendar.type == 'Holiday') & (calendar.transferred == True ), 'holiday_or_weekday'] = 'weekday'
calendar = calendar[['date', 'dcoilwtico','day_week', 'holiday_or_weekday']]

In [14]:
calendar.head()

,date,dcoilwtico,day_week,holiday_or_weekday
0,2013-01-01,93.14,1,holiday
1,2013-01-02,93.14,2,weekday
2,2013-01-03,92.97,3,weekday
3,2013-01-04,93.12,4,weekday
4,2013-01-05,93.12,5,weekday


In [15]:
data["date"] = pd.to_datetime(data["date"])
df = pd.merge(data, calendar, how='left', on='date')

In [16]:
df.head()

,id,date,store_nbr,family,onpromotion,dcoilwtico,day_week,holiday_or_weekday
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.8,2,weekday
1,3000889,2017-08-16,1,BABY CARE,0,46.8,2,weekday
2,3000890,2017-08-16,1,BEAUTY,2,46.8,2,weekday
3,3000891,2017-08-16,1,BEVERAGES,20,46.8,2,weekday
4,3000892,2017-08-16,1,BOOKS,0,46.8,2,weekday


In [17]:
df = pd.merge(df, stores, how='left', on='store_nbr')

In [18]:
df.head()

,id,date,store_nbr,family,onpromotion,dcoilwtico,day_week,holiday_or_weekday,city,state,type,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,46.8,2,weekday,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,46.8,2,weekday,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,46.8,2,weekday,Quito,Pichincha,D,13
3,3000891,2017-08-16,1,BEVERAGES,20,46.8,2,weekday,Quito,Pichincha,D,13
4,3000892,2017-08-16,1,BOOKS,0,46.8,2,weekday,Quito,Pichincha,D,13


In [20]:
df = df[["id", "date", "store_nbr", "city", "state", "type", "cluster", "family", "onpromotion", "dcoilwtico", "day_week", "holiday_or_weekday"]]

In [21]:
df.head()

,id,date,store_nbr,city,state,type,cluster,family,onpromotion,dcoilwtico,day_week,holiday_or_weekday
0,3000888,2017-08-16,1,Quito,Pichincha,D,13,AUTOMOTIVE,0,46.8,2,weekday
1,3000889,2017-08-16,1,Quito,Pichincha,D,13,BABY CARE,0,46.8,2,weekday
2,3000890,2017-08-16,1,Quito,Pichincha,D,13,BEAUTY,2,46.8,2,weekday
3,3000891,2017-08-16,1,Quito,Pichincha,D,13,BEVERAGES,20,46.8,2,weekday
4,3000892,2017-08-16,1,Quito,Pichincha,D,13,BOOKS,0,46.8,2,weekday


In [22]:
df.to_csv("datasets/features/submit/data.csv", index=False)

In [23]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ["city", "state", "type", "family", "holiday_or_weekday"]
le = LabelEncoder()
for column in columns_to_encode:
    df[column] = le.fit_transform(df[column])
    
df.head()

,id,date,store_nbr,city,state,type,cluster,family,onpromotion,dcoilwtico,day_week,holiday_or_weekday
0,3000888,2017-08-16,1,18,12,3,13,0,0,46.8,2,1
1,3000889,2017-08-16,1,18,12,3,13,1,0,46.8,2,1
2,3000890,2017-08-16,1,18,12,3,13,2,2,46.8,2,1
3,3000891,2017-08-16,1,18,12,3,13,3,20,46.8,2,1
4,3000892,2017-08-16,1,18,12,3,13,4,0,46.8,2,1


In [25]:
df.to_csv("datasets/features/submit/transformed.csv", index=False)